<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.61
    Uninstalling yfinance-0.2.61:
      Successfully uninstalled yfinance-0.2.61
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))
  print('-------------------')
  print('Est LTT: ',  format_amt(estimate_lt))
  print('Est LTT PnL: '+ '{} ({}%)'.format(format_amt(est_lt_pnl_amount), est_lt_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')

Run date time (IST): 2025-06-03 11:04:06
-------------------
qualified stocks: 93
with latest results: 32
still star stocks: 18
-------------------
Initial Investment:  1.17 C
CY Investment:  1.44 C
Reserve:  4.24 L
Current:  1.33 C
-------------------
Today PnL: 13.68 K (0.1%)
Current PnL: -15.73 L (-10.96%)
CY Booked + Current PnL: -11.26 L (-7.85%)
-------------------
Total profit:  4.47 L
Total loss:  -20.21 L
-------------------
Total Booked + Current PnL: 15.72 L (13.49%)
Total Booked PnL: 31.45 L (26.99%)
Curr Year Booked PnL: 4.47 L (3.37%)
Prev Year Booked PnL: 26.98 L (23.15%)
Est FTT:  2.11 C
Est FTT PnL: 77.93 L (58.74%)
-------------------
Est LTT:  2.73 C
Est LTT PnL: 1.40 C (105.58%)
Deployed:  1.17 C
Current:  1.33 C
CAGR/XIRR %: 9.27%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,PIDILITIND,2504.06,3202.0,-16.12,X-LC,36.56,92973.0,17851.0,3087.0,0.61,23.76,3.32,27.87,5.0,5.78,0.72,16.60,X40,NTT,CHEMICALS
49,LICI,785.05,983.0,-2.96,H-LC,88.17,237782.0,43090.0,5992.0,-0.37,22.13,2.52,25.21,31.0,7.19,1.85,31.49,XY25,NTT,INSURANCE
85,UNIONBANK,123.87,163.0,1.23,M-LC,94.62,177770.0,36930.0,7555.0,2.08,26.22,4.25,31.59,78.0,4.89,1.38,54.42,XY24,NTT,BANKS
4,ADANIPORTS,1103.69,1583.0,2.14,M-LC,59.14,102439.0,24077.0,9957.0,-1.72,30.73,9.72,43.43,70.0,2.42,0.80,37.01,XY24,NTT,MISC
62,RELIANCE,1291.83,1526.0,3.85,X-LC,16.13,154180.0,13371.0,12149.0,0.06,9.50,7.88,18.13,19.0,1.10,1.20,21.74,XY25,NTT,REFINERIES


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
67,SATIN,199.77,274.00,-21.16,H-SC,12.90,156519.0,-39455.0,112271.0,3.66,-20.13,71.73,37.16,124.0,-0.35,1.22,16.89,XY24,NTT,FINANCE
38,INDIAMART,2327.09,4911.36,-51.47,H-SC,55.38,126522.0,3186.0,133784.0,2.72,2.58,105.74,111.05,108.0,0.02,0.99,23.97,AR,ATH,MISC
47,LALPATHLAB,2804.19,3545.00,-32.18,H-SC,40.86,155964.0,1734.0,39007.0,2.52,1.12,25.01,26.42,114.0,0.04,1.21,22.73,X40N,NTT,HEALTHCARE
85,UNIONBANK,123.87,163.00,1.23,M-LC,94.62,177770.0,36930.0,7555.0,2.08,26.22,4.25,31.59,78.0,4.89,1.38,54.42,XY24,NTT,BANKS
66,SAMMAANCAP,170.35,326.00,-162.93,M-SC,49.46,75834.0,-26376.0,119765.0,2.06,-25.81,157.93,91.37,179.0,-0.22,0.59,23.10,XY25,NTT,FINANCE


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
4,ADANIPORTS,1103.69,1583.00,2.14,M-LC,59.14,102439.0,24077.0,9957.0,-1.72,30.73,9.72,43.43,70.0,2.42,0.80,37.01,XY24,NTT,MISC
19,COALINDIA,406.65,512.15,5.34,L-LC,38.71,145817.0,-4644.0,43672.0,-1.40,-3.09,29.95,25.94,145.0,-0.11,1.14,11.26,XY25,ATH,MINING
37,ICICIPRULI,600.83,791.05,-8.38,H-MC,58.06,148007.0,12219.0,30771.0,-1.39,9.00,20.79,31.66,105.0,0.40,1.15,22.05,X40,ATH,INSURANCE
34,HINDUNILVR,2423.50,2723.00,-16.14,X-LC,24.73,227969.0,-7110.0,36156.0,-0.90,-3.02,15.86,12.36,7.0,-0.20,1.77,8.94,XY25,NTT,FMCG
63,REPCOHOME,515.07,880.00,-56.78,H-SC,35.48,159140.0,-45858.0,191095.0,-0.89,-22.37,120.08,70.85,121.0,-0.24,1.24,28.47,XY24,NTT,FINANCE


In [12]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
41,INFY,1450.31,2275.00,-13.50,X-LC,45.70,246370.0,15771.0,115350.0,-0.28,6.84,46.82,56.86,1.0,0.14,1.92,14.03,X40,BTT,IT
80,TCS,3841.06,4998.00,-17.76,X-LC,19.35,236980.0,-28053.0,107873.0,-0.44,-10.58,45.52,30.12,2.0,-0.26,1.85,6.28,X40,BTT,IT
50,LTIM,5564.16,7197.33,-5.69,X-LC,68.82,181764.0,-18546.0,77341.0,-0.46,-9.26,42.55,29.35,3.0,-0.24,1.42,25.57,X200,ATH,IT
56,PIDILITIND,2504.06,3202.00,-16.12,X-LC,36.56,92973.0,17851.0,3087.0,0.61,23.76,3.32,27.87,5.0,5.78,0.72,16.60,X40,NTT,CHEMICALS
6,ASIANPAINT,2961.56,4250.00,-22.73,X-LC,4.30,153714.0,-47672.0,135284.0,-0.27,-23.67,88.01,43.51,6.0,-0.35,1.20,6.02,X40,BTT,PAINTS


In [13]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,PFIZER,4245.50,6318.3,13.02,H-SC,98.92,130652.0,33006.0,14672.0,1.01,33.80,11.23,48.82,134.0,2.25,1.02,45.14,X40,ATH,PHARMA
85,UNIONBANK,123.87,163.0,1.23,M-LC,94.62,177770.0,36930.0,7555.0,2.08,26.22,4.25,31.59,78.0,4.89,1.38,54.42,XY24,NTT,BANKS
4,ADANIPORTS,1103.69,1583.0,2.14,M-LC,59.14,102439.0,24077.0,9957.0,-1.72,30.73,9.72,43.43,70.0,2.42,0.80,37.01,XY24,NTT,MISC
49,LICI,785.05,983.0,-2.96,H-LC,88.17,237782.0,43090.0,5992.0,-0.37,22.13,2.52,25.21,31.0,7.19,1.85,31.49,XY25,NTT,INSURANCE
56,PIDILITIND,2504.06,3202.0,-16.12,X-LC,36.56,92973.0,17851.0,3087.0,0.61,23.76,3.32,27.87,5.0,5.78,0.72,16.60,X40,NTT,CHEMICALS


In [14]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SAIL,130.64,228.00,100.16,M-MC,84.95,227734.0,2772.0,164879.0,1.19,1.23,72.40,74.53,170.0,0.02,1.77,32.22,XY24,BTT,STEEL
51,LUXIND,1448.19,2442.00,38.95,M-SC,64.52,144001.0,6423.0,87985.0,-0.58,4.67,61.10,68.62,183.0,0.07,1.12,24.84,X40N,NTT,TEXTILES
55,PFIZER,4245.50,6318.30,13.02,H-SC,98.92,130652.0,33006.0,14672.0,1.01,33.80,11.23,48.82,134.0,2.25,1.02,45.14,X40,ATH,PHARMA
28,GILLETTE,8109.44,10330.69,10.66,H-SC,90.32,163174.0,25314.0,12450.0,1.37,18.36,7.63,27.39,116.0,2.03,1.27,36.53,X40,ATH,FMCG
57,POLYCAB,5005.95,7541.35,6.01,H-MC,37.63,100614.0,15513.0,27588.0,-0.14,18.23,27.42,50.65,90.0,0.56,0.78,26.48,X40N,ATH,CABLES


In [15]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
17,CAMS,3643.00,5250.99,4.44,H-SC,69.89,117496.0,15492.0,29527.0,1.38,15.19,25.13,44.14,111.0,0.52,0.91,33.90,XR,ATH,MISC
11,BANKINDIA,113.49,190.00,-27.01,M-MC,62.37,143695.0,14543.0,72523.0,0.81,11.26,50.47,67.42,156.0,0.20,1.12,39.76,XR,NTT,BANKS
38,INDIAMART,2327.09,4911.36,-51.47,H-SC,55.38,126522.0,3186.0,133784.0,2.72,2.58,105.74,111.05,108.0,0.02,0.99,23.97,AR,ATH,MISC
92,WIPRO,243.46,420.00,-10.62,M-LC,32.26,153066.0,2121.0,107330.0,-0.32,1.41,70.12,72.51,68.0,0.02,1.19,14.93,XR,NTT,IT
1,AARTIIND,487.04,919.00,39.14,M-SC,53.76,79884.0,-1452.0,73589.0,1.64,-1.78,92.12,88.69,205.0,-0.02,0.62,34.71,XR,NTT,CHEMICALS


In [16]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
52,MASFIN,326.60,399.5,-21.86,H-SC,73.12,88860.0,-9120.0,30994.0,0.71,-9.31,34.88,22.32,125.0,-0.29,0.69,28.88,XR,ATH,FINANCE
4,ADANIPORTS,1103.69,1583.0,2.14,M-LC,59.14,102439.0,24077.0,9957.0,-1.72,30.73,9.72,43.43,70.0,2.42,0.80,37.01,XY24,NTT,MISC
84,UJJIVANSFB,52.77,60.0,37.69,M-SC,56.99,119610.0,-22869.0,42390.0,-0.43,-16.05,35.44,13.70,221.0,-0.54,0.93,41.17,X40N,NTT,BANKS
87,VALIANTORG,512.64,838.0,-380.72,H-SC,100.00,119392.0,-13894.0,98486.0,-0.04,-10.42,82.49,63.47,131.0,-0.14,0.93,100.13,XR,NTT,CHEMICALS
55,PFIZER,4245.50,6318.3,13.02,H-SC,98.92,130652.0,33006.0,14672.0,1.01,33.80,11.23,48.82,134.0,2.25,1.02,45.14,X40,ATH,PHARMA


In [17]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
87,VALIANTORG,512.64,838.00,-380.72,H-SC,100.00,119392.0,-13894.0,98486.0,-0.04,-10.42,82.49,63.47,131.0,-0.14,0.93,100.13,XR,NTT,CHEMICALS
55,PFIZER,4245.50,6318.30,13.02,H-SC,98.92,130652.0,33006.0,14672.0,1.01,33.80,11.23,48.82,134.0,2.25,1.02,45.14,X40,ATH,PHARMA
7,ASIANTILES,75.41,137.00,6566.67,M-SC,97.85,73147.0,-20663.0,97278.0,-0.31,-22.03,132.99,81.67,214.0,-0.21,0.57,41.65,XR,NTT,CERAMICS
74,STARHEALTH,551.56,761.00,5.64,M-SC,96.77,172606.0,-27610.0,103633.0,0.30,-13.79,60.04,37.97,196.0,-0.27,1.34,38.59,XY24,NTT,INSURANCE
77,TANLA,917.30,1963.11,-33.79,H-SC,95.70,162551.0,-76864.0,349826.0,-0.82,-32.11,215.21,114.01,113.0,-0.22,1.27,49.08,AR,ATH,IT


In [18]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,18.78
1,25,41.17
2,50,70.03


In [19]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24    32.17
XY25    14.36
X40     14.26
X40N    13.56
AR      12.46
XR      10.61
X200     1.42
SR       1.16
Name: CurrAlloc%, dtype: float64

In [20]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    21.72
M-SC    17.81
X-LC    17.26
H-MC     8.67
M-LC     7.86
H-LC     7.29
X-MC     5.64
M-MC     5.62
X-SC     2.80
L-SC     2.34
L-LC     1.14
Name: CurrAlloc%, dtype: float64

In [21]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,13.41,-0.44,32.17
BANKS,9.98,-9.23,59.45
IT,9.88,-18.38,83.08
MISC,6.53,-29.18,86.32
FINANCE,6.01,-23.89,82.71
INSURANCE,5.41,4.19,24.52
PAINTS,5.38,-19.19,54.46
ELECTRICAL,5.01,-4.64,50.56
AUTO,4.40,-10.70,59.06


In [22]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2841318.0,26
X40N,1204472.0,17
AR,1168423.0,12
XR,1089271.0,14
X40,759002.0,11
XY25,457522.0,10
SR,195876.0,2
X200,77341.0,1


In [23]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2227829.0,21
M-SC,2027182.0,21
X-LC,794625.0,12
M-MC,545051.0,5
H-MC,514078.0,8
X-SC,348421.0,4
X-MC,308633.0,5
L-SC,297533.0,4
M-LC,296550.0,6


In [24]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      930993.0      6
M-SC       XY24      747552.0      6
H-SC       AR        542910.0      3
           X40N      494166.0      6
M-SC       X40N      467323.0      6
X-LC       X40       448578.0      5
M-MC       XY24      390614.0      3
M-SC       XR        291505.0      4
H-SC       XR        232638.0      4
M-SC       X40       222269.0      2
H-LC       AR        181940.0      3
M-LC       XY24      176132.0      4
H-MC       XY24      174152.0      2
L-SC       XR        156081.0      2
X-LC       AR        143803.0      2
X-MC       XY24      139502.0      2
X-LC       XY25      124903.0      4
X-SC       X40N      121172.0      2
M-SC       XY25      119765.0      1
X-SC       XR        116049.0      1
H-MC       XR        113145.0      1
X-SC       SR        111200.0      1
M-LC       XR        107330.0      1
X-MC       X40N       94223.0      2
M-SC       AR         94092.0      1
           SR         84676.0      1
M-MC       AR         81914.0      1
L-SC       XY24       80654.0      1
X-LC       X200       77341.0      1
H-MC       XY25       75194.0      1
X-MC       XY25       74908.0      1
M-MC       XR         72523.0      1
H-MC       AR         62966.0      1
           X40        61033.0      2
L-SC       AR         60798.0      1
L-LC       XY25       43672.0      1
H-MC       X40N       27588.0      1
H-SC       X40        27122.0      2
M-LC       XY25       13088.0      1
H-LC       XY25        5992.0      1

In [25]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 29.0 seconds
